In [0]:
#数据分析库
import pandas as pd
#科学计算库
import numpy as np 
from pandas import Series,DataFrame
#引入数据
data_train = pd.read_csv("datalab/1386/titanic_train.csv")
data_test = pd.read_csv("datalab/1386/titanic_test.csv")

#Age列中的缺失值用Age中位数进行填充
data_train["Age"] = data_train['Age'].fillna(data_train['Age'].median())  
data_train.describe()

#线性回归
from sklearn.linear_model import LinearRegression   

#训练集交叉验证，得到平均值
#from sklearn.cross_validation import KFold 
from sklearn.model_selection import KFold
 
#选取简单的可用输入特征
predictors = ["Pclass","Age","SibSp","Parch","Fare"]     
 
#初始化线性回归算法
alg = LinearRegression()
#样本平均分成3份，3折交叉验证
#kf = KFold(data_train.shape[0],n_folds=3,random_state=1)   
kf = KFold(n_splits=3,shuffle=False,random_state=1) 

predictions = []
for train,test in kf.split(data_train):
	#The predictors we're using to train the algorithm.  Note how we only take then rows in the train folds.
	train_predictors = (data_train[predictors].iloc[train,:])
	#The target we're using to train the algorithm.
	train_target = data_train["Survived"].iloc[train]
	#Training the algorithm using the predictors and target.
	alg.fit(train_predictors,train_target)
	#We can now make predictions on the test fold
	test_predictions = alg.predict(data_train[predictors].iloc[test,:])
	predictions.append(test_predictions)

import numpy as np
 
#The predictions are in three aeparate numpy arrays.	Concatenate them into one.
#We concatenate them on axis 0,as they only have one axis.
predictions = np.concatenate(predictions,axis=0)
 
#Map predictions to outcomes(only possible outcomes are 1 and 0)
predictions[predictions>.5] = 1
predictions[predictions<=.5] = 0
accuracy = sum(predictions == data_train["Survived"]) / len(predictions)
print ("准确率为: ", accuracy)

from sklearn import model_selection
#逻辑回归
from sklearn.linear_model import LogisticRegression   

#初始化逻辑回归算法
#LogRegAlg=LogisticRegression(random_state=1)
#liblinear
LogRegAlg=LogisticRegression(solver='liblinear')
re = LogRegAlg.fit(data_train[predictors],data_train["Survived"])

#使用sklearn库里面的交叉验证函数获取预测准确率分数
scores = model_selection.cross_val_score(LogRegAlg,data_train[predictors],data_train["Survived"],cv=3)

#使用交叉验证分数的平均值作为最终的准确率
print("准确率为: ",scores.mean())

#Sex性别列处理：male用0，female用1
data_train.loc[data_train["Sex"] == "male","Sex"] = 0
data_train.loc[data_train["Sex"] == "female","Sex"] = 1

#缺失值用最多的S进行填充
data_train["Embarked"] = data_train["Embarked"].fillna('S') 
#地点用0,1,2
data_train.loc[data_train["Embarked"] == "S","Embarked"] = 0    
data_train.loc[data_train["Embarked"] == "C","Embarked"] = 1
data_train.loc[data_train["Embarked"] == "Q","Embarked"] = 2

predictors = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]  

#LogRegAlg=LogisticRegression(random_state=1)
LogRegAlg=LogisticRegression(solver='liblinear')

#Compute the accuracy score for all the cross validation folds.(much simpler than what we did before!)
re = LogRegAlg.fit(data_train[predictors],data_train["Survived"])
scores = model_selection.cross_val_score(LogRegAlg,data_train[predictors],data_train["Survived"],cv=3)
#Take the mean of the scores (because we have one for each fold)
print("准确率为: ",scores.mean())